需求说明：
    找出polymarket上 strike date的3天前和7天前市场发生概率大于0.9，小于0.95，且在strike date 当日发生成交的概率

分解需求：
    1、从文档(polymarket\markets_data.csv) 找出 已经关闭的事件，字段closed
    2、获取已关闭事件的所有市场的token_id。并获取历史价格数据，全部存入对应question_id
    3、找到在 strike date 之前的倒数第三天和倒数第7天的市场发生概率大于0.9且小于0.95的事件及其市场，计算总的事件数N
    4、找到第3、的市场，看其是否在strike date 当日的概率为1。计算win的总事件的数目m
    5、计算总发生概率 m/N，并输出总结果

In [1]:
import pandas as pd
import numpy as np
import csv
import json
import os
from py_clob_client.client import ClobClient,BookParams
from py_clob_client.constants import POLYGON
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

import requests
from datetime import datetime
import re

1、从文档('polymarket\markets_data.csv') 找出已经关闭的事件，字段closed

In [2]:
poly_data = pd.read_csv('polymarket\markets_data.csv')

In [4]:
poly_data.columns

Index(['accepting_order_timestamp', 'accepting_orders', 'active', 'archived',
       'closed', 'condition_id', 'description', 'enable_order_book',
       'end_date_iso', 'fpmm', 'game_start_time', 'icon', 'image',
       'is_50_50_outcome', 'maker_base_fee', 'market_slug',
       'minimum_order_size', 'minimum_tick_size', 'neg_risk',
       'neg_risk_market_id', 'neg_risk_request_id', 'notifications_enabled',
       'question', 'question_id', 'rewards', 'seconds_delay', 'tags',
       'taker_base_fee', 'token_outcome', 'token_price', 'token_token_id',
       'token_winner', 'tokens'],
      dtype='object')

In [6]:
# poly_data['token_winner']

In [7]:
poly_data_closed=poly_data[poly_data['closed']==True][['condition_id','question','question_id','tokens','token_winner']].reset_index(drop=True)

2、获取已关闭事件的所有市场的token_id，并获取历史价格数据，全部存入对应question_id

In [132]:
clob_endpoint = "https://clob.polymarket.com"
url = f"{clob_endpoint}/prices-history" # 获取历史价格

In [260]:
i

13519

In [258]:
js_str

'[{"token_id": "75674010897912433263470070979110319954823328111562242389419248656674366188074", "outcome": "Gillen ", "price": 1, "winner": true}, {"token_id": "23682917493470547014927814926256831892437243307768785697446430339870292046840", "outcome": "D"Esposito", "price": 0, "winner": false}]'

In [ ]:
pattern = r'(?<=[a-zA-Z0-9])"s'

re.sub(pattern,'',)

In [264]:
len(tokens_id)

2

In [94]:
# pro_path = '2024-11-15/price_history/'
# questions_id = []
# for i in range(poly_data_closed.shape[0]):
#     tmp_data = poly_data_closed.iloc[i,:]
#     que_id = tmp_data['question_id']
#     os.makedirs(pro_path+str(que_id), exist_ok=True)

#     questions_id.append(que_id)

#     # 获取token_id
#     js_str = tmp_data['tokens'].replace("'",'"')
#     if 'False' in js_str:  # 如果str中包含False 或者 True, 必须转化为小写
#         js_str=js_str.replace('False','false')

#     if 'True' in js_str:
#         js_str=js_str.replace('True','true')

#     # json字符串的处理1
#     pattern = r'(?<=[a-zA-Z0-9])"s'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
#     js_str = re.sub(pattern, '', js_str)    

#     # json字符串的处理2
#     pattern = r'(?<=[a-zA-Z0-9])"N'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
#     js_str = re.sub(pattern, '', js_str) 

#     # json字符串的处理3
#     pattern = r'(?<=[a-zA-Z0-9])"E'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
#     js_str = re.sub(pattern, '', js_str)  

#     tokens_id=json.loads(js_str)

#     if len(os.listdir(pro_path+str(que_id)))==len(tokens_id):
#         continue

#     # 获取历史价格数据
#     for j in range(len(tokens_id)):
#         token_id = tokens_id[j]['token_id']
#         params = {
#             "market": token_id,  # 替换为实际的CLOB token id
#             "interval": "max",  # 获取最大长度的数据
#             "fidelity": 60*24  # 数据分辨率，单位为分钟
#         }
#         try:
#             # 发送GET请求
#             response = requests.get(url, params=params)
#             response.raise_for_status()  # 如果响应状态码不是200，将抛出异常
#             # 解析响应数据
#             prices_history = response.json()
#             prices_history = pd.DataFrame(prices_history['history'])
#             prices_history['t'] = prices_history['t'].apply(lambda x:datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
#             file_path = f'{pro_path}/{que_id}/{token_id}.csv'
#             # print(file_path)
#             prices_history.to_csv(file_path, index=False)
#         except requests.RequestException as e:
#             print(f"请求错误：{e}")
#             print(i,que_id)
        
        

3、找到在 strike date 前 的倒数第三天和倒数第7天的市场发生概率大于0.9且小于0.95的事件及其市场，计算总的事件数N

In [225]:
# pro_path = '2024-11-15/limited_events/'
events_id = [] # 存入满足条件的 question_id
markets_id = [] # 存入满足条件的 token_id
N = 0 # 记录满足条件的总事件数目
flag = False  # flag = False 表示该事件的市场中没有满足条件的市场
for file_path in os.listdir('2024-11-15/price_history/'):
    que_id = file_path

    for f_p in os.listdir(f'2024-11-15/price_history/{que_id}'):
        prices_history = pd.read_csv(f'2024-11-15/price_history/{que_id}/{f_p}')

        if prices_history.shape[0]>=8:
            # strike date 前 倒数第三天的价格
            pre_3d = prices_history.iloc[-4,:]
            price_pre_3d = pre_3d['p']

            # strike date 前 倒数第7天的价格
            pre_7d = prices_history.iloc[-8,:]
            price_pre_7d = pre_7d['p']

            if 0.9<price_pre_3d<0.95 and 0.9<price_pre_7d<0.95:
                markets_id.append(f_p[:-4])
                flag=True
    
    if flag == True:
        N+=1
        events_id.append(que_id)
    
    flag=False



4、找到第3、的市场，看其是否在strike date 当日的概率为1。计算win的总事件的数目m

In [226]:
# 为确保准确，应该找到交易结束的时间

m=0 # 计算win的事件个数
m_events_id = [] # 存储win的 question_id
m_markets_id = [] # 存储win的 市场id
for i in range(len(events_id)):
    que_id = events_id[i]
    tmp_data = poly_data_closed[poly_data_closed['question_id']==que_id]
    if not tmp_data.empty:
    
        js_str = tmp_data['tokens'].values[0].replace("'",'"')
        if 'False' in js_str:  # 如果str中包含False 或者 True, 必须转化为小写
            js_str=js_str.replace('False','false')

        if 'True' in js_str:
            js_str=js_str.replace('True','true')

        # # json字符串的处理1
        # pattern = r'(?<=[a-zA-Z0-9])"s'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
        # js_str = re.sub(pattern, '', js_str)    

        # # json字符串的处理2
        # pattern = r'(?<=[a-zA-Z0-9])"N'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
        # js_str = re.sub(pattern, '', js_str) 

        # # json字符串的处理3
        # pattern = r'(?<=[a-zA-Z0-9])"E'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
        # js_str = re.sub(pattern, '', js_str)  

        tokens_id=json.loads(js_str)

        # 获取win是否为True
        for j in range(len(tokens_id)):
            if tokens_id[j]['token_id']==markets_id[i] and tokens_id[j]['winner']==True:
                m+=1
                m_events_id.append(que_id)
                m_markets_id.append(markets_id[i])
            

5、计算总发生概率 m/N

In [227]:
m

116

In [228]:
N

177

In [229]:
m/N

0.655367231638418

6、输出上述表述的事件及市场

In [230]:
final_data = pd.DataFrame()
for i in range(len(m_events_id)):
    que_id = m_events_id[i]
    data_tmp = poly_data[poly_data['question_id']==que_id]

    que = data_tmp['question'].values[0]

    js_str = data_tmp['tokens'].values[0].replace("'",'"')
    if 'False' in js_str:  # 如果str中包含False 或者 True, 必须转化为小写
            js_str=js_str.replace('False','false')
    if 'True' in js_str:
        js_str=js_str.replace('True','true')

    token_dict = json.loads(js_str)
    token_frame = pd.DataFrame(token_dict)
    token_frame=token_frame[token_frame['token_id']==m_markets_id[i]]
    
    final_data.loc[i,'question'] = que
    final_data.loc[i,'outcome'] = token_frame['outcome'].values[0]
    final_data.loc[i,'price'] = token_frame['price'].values[0]
    final_data.loc[i,'winner'] = token_frame['winner'].values[0]

    # 开始和结束时间
    df = pd.read_csv(f'2024-11-15/price_history/{que_id}/{m_markets_id[i]}.csv')
    start_time = df['t'].values[0]
    end_time = df['t'].values[-1]

    final_data.loc[i,'start_time'] = start_time
    final_data.loc[i,'end_time'] = end_time


In [231]:
final_data

,question,outcome,price,winner,start_time,end_time
0,Will U.S. inflation be greater than 0.0% from ...,Yes,1.0,True,2023-06-13 08:00:02,2023-07-12 08:00:02
1,Will Sam Altman join Grok by Dec 31?,No,1.0,True,2023-11-19 08:00:03,2024-01-01 08:00:02
2,Jupiter airdrop in 2023?,No,1.0,True,2023-12-12 08:00:02,2024-01-01 08:00:02
3,US adds between 250k and 300k jobs in August?,No,1.0,True,2024-08-03 08:00:02,2024-09-06 08:00:02
4,US adds more than 300k jobs in August?,No,1.0,True,2024-08-03 08:00:02,2024-09-06 08:00:02
...,...,...,...,...,...,...
111,Will Sinema announce she's running for re-elec...,No,1.0,True,2024-02-01 08:00:02,2024-03-05 08:00:03
112,Blast airdrop by May?,No,1.0,True,2023-11-25 08:00:02,2024-06-01 08:00:03
113,Will Nigel Farage win election to UK parliament?,Yes,1.0,True,2024-06-04 08:00:02,2024-07-05 08:00:02
114,Will X remove likes + reposts counter in March?,No,1.0,True,2024-03-08 08:00:02,2024-04-01 08:00:02


In [233]:
final_data.to_csv('2024-11-15/results.csv',index=False)

In [234]:
# 计算总样本数
pro_path = '2024-11-15/price_history/'
X=0
for i in os.listdir(pro_path):
    if len(os.listdir(pro_path+'/'+i))>=2:
        X+=1

In [235]:
X

12642

,accepting_order_timestamp,accepting_orders,active,archived,closed,condition_id,description,enable_order_book,end_date_iso,fpmm,...,question_id,rewards,seconds_delay,tags,taker_base_fee,token_outcome,token_price,token_token_id,token_winner,tokens
